In [133]:
# Padé Approximants

import numpy as np
from scipy.linalg import solve
from scipy.special import factorial

def pade_approximant(series_coefficients, L, M):
    
    if len(series_coefficients) < L + M + 1:
        raise ValueError("Insufficient number of coefficients for chosen L and M.")

    A = np.zeros((M, M))
    for i in range(M):
        for j in range(M):
            A[i, j] = series_coefficients[L + i - j]

    b = -np.array(series_coefficients[L + 1 : L + M + 1])

    q = np.zeros(M + 1)
    q[1:] = solve(A, b)
    q[0] = 1

    p = np.zeros(L + 1)
    for k in range(L + 1):
        p[k] = sum(
            series_coefficients[k - j] * q[j]
            for j in range(min(k + 1, M + 1))
        )

    P = np.poly1d(p[::-1])
    Q = np.poly1d(q[::-1])

    return P, Q


# Example 1: e^x series coefficients (1, 1, 1/2!, 1/3!, ...)
series_coeffs = [1 / factorial(i) for i in range(10)]

# Pade Approximant with L=1, M=1
L, M = 1, 1
P, Q = pade_approximant(series_coeffs, L, M)
print(f"P(x): {P}")
print(f"Q(x): {Q}\n\n")

# Example 2: 
series_coeffs = [1, 1, 0, 0, -1, 1, -1, 2, -3, 4]
L, M = 5, 3
P, Q = pade_approximant(series_coeffs, L, M)
print(f"P(x): {P}")
print(f"Q(x): {Q}\n\n")


P(x):  
0.5 x + 1
Q(x):  
-0.5 x + 1


P(x):    3     2
1 x + 1 x + 2 x + 1
Q(x):    3
1 x + 1 x + 1




In [123]:
# C-Table

import numpy as np
from scipy.linalg import det
from math import factorial
import pandas as pd


def generate_c_table(series_coefficients, max_L, max_M):

    C = np.zeros((max_L + 1, max_M + 1), dtype=float)

    # Rule 1: C(L/0) = 1
    for L in range(max_L + 1):
        C[L, 0] = 1

    # Rule 2: C(L/1) = c_L
    for L in range(len(series_coefficients)):
        if L <= max_L:
            C[L, 1] = series_coefficients[L]

    # Rule 3: C(0/M) = (-1)^(M(M-1)/2) * c_0^M
    c_0 = series_coefficients[0]
    for M in range(1, max_M + 1):
        C[0, M] = (-1) ** (M * (M - 1) // 2) * (c_0 ** M)

    # Rule 4: Frobenius formula
    for L in range(1, max_L + 1):
        for M in range(2, max_M + 1):
            if L + 1 <= max_L and L - 1 >= 0 and M - 2 >= 0 and C[L, M - 1] != 0:
                C[L, M] = (
                    C[L + 1, M - 1] * C[L - 1, M - 1] - C[L, M - 1] ** 2
                ) / C[L, M - 2]
            else:
                # If the recursive formula is not applicable, compute determinant
                if L - M + 1 >= 0:
                    rows = M
                    cols = M
                    matrix = np.zeros((rows, cols))
                    for i in range(rows):
                        for j in range(cols):
                            matrix[i, j] = series_coefficients[L - M + 1 + i + j]
                    C[L, M] = det(matrix)

    return C



# Power series of f(z) = 1 + z - z^4 + z^5 - z^6 + 2z^7 - 3z^8 + 4z^9 - ...
series_coeffs = [1, 1, 0, 0, -1, 1, -1, 2, -3, 4]

max_L, max_M = 6, 3
c_table = generate_c_table(series_coeffs, max_L, max_M).transpose()

c_table_df = pd.DataFrame(
    c_table, 
    columns=[f"L={i}" for i in range(max_L + 1)],
    index=[f"M={j}" for j in range(max_M + 1)]
)
c_table_df = c_table_df.round(4)  # Limit to 4 decimal places

# Print the DataFrame
print(c_table_df)


     L=0  L=1  L=2  L=3  L=4  L=5  L=6
M=0  1.0  1.0  1.0  1.0  1.0  1.0  1.0
M=1  1.0  1.0  0.0  0.0 -1.0  1.0 -1.0
M=2 -1.0 -1.0  0.0  0.0 -1.0  0.0  1.0
M=3 -1.0 -1.0  1.0 -1.0  1.0 -1.0  1.0
